- https://github.com/aamini/introtodeeplearning/
- https://github.com/pytorch/examples/tree/master/word_language_model
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://github.com/karpathy/char-rnn

# LSTM

<img src="https://miro.medium.com/max/4136/1*SKGAqkVVzT6co-sZ29ze-g.png" width="70%"> 

- RNN učimo generirati tekst
- slučaj 1: "_na <span style="color:blue">nebu</span> su <span style="color:blue">oblaci</span>"
- slučaj 2: "_Rodio sam se u <span style="color:blue">Francuskoj</span>. Tamo sam živio do desete godine. Stoga tečno pričam <span style="color:blue">francuski</span>._"
- razmak između bitnih informacija postaje velik

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png" width="60%"> 

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png" width="60%"> 


- LSTM (Long Short Term Memory) su varijanta RNN
- imaju mogućnost učiti long-term dependecies

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" width="60%"> 

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" width="60%"> 

- horizontalna linija koja prenosi informacije
- vrata (gates) koja propuštaju ili ne informaciju
- vrata su sigmoid + $\times$
- sigmoid funkcija vrati postotak koliko informacije propustiti
1. Prva odluka: koju informaciju propustiti iz prošlog stanja: *forget gate*
    - $f_t = \sigma(W_f [h_{t-1},x_t] + b_f)$
2. Druga odluka: koju novu informaciju uzeti:
    - sigmoid sloj: koje vrijednosti ćemo ažurirati
    - $i_t = \sigma(W_i[h_{t-1},x_t] + b_i)$
    - tanh sloj: kreira novi vektor koji je kandidat za sljedeće stanje
    - $\hat{C}_t = \tanh{(W_c[h_{t-1},x_t]+ b_c)}$
3. Ažuriranje stanja $C_{t-1} \rightarrow C_t$
    - $C_t = f_t\cdot C_{t-1} + i_t\cdot  \hat{C}_t$
4. Odluka o izlazu
    - $o_t = \sigma(W_0[h_{t-1},x_t] + b_o)$
    - $h_t = o_t\cdot \tanh{C_t}$
    
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" width="60%">


## Zadatak:

Po uzoru na Karpathyjev model RNN-a koji je naučen generirati stihove s obzirom na Shakespeare-ov korpus, napravite model koji će naučiti generirati stihove s obzirom na Cesarićev korpus. 

In [257]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import math
import re
from datetime import datetime

Pogledajmo srednju vrijednost broja svih riječi i broja paragrafa (pjesama)

In [416]:
with open('./cesaric/train.txt', 'r', encoding = 'utf-8') as outf:
    paragraf = []
    counter = 0
    total_count_paragrafs = 0
    sum_words = 0
    for line in outf:
        if line == '\n': 
            total_count_paragrafs += 1
            sum_words += len(paragraf)
            paragraf = []
        else:
            if line.count('-') >= 3:
                continue
            else:
                paragraf += re.findall(r"[\w]+", line)
        counter += 1
            
    print("U prosjeku, Cesarićeva pjesma ima: {} riječi u jednoj PJESMI.".format(np.round(sum_words / total_count_paragrafs)))
    print("U prosjeku, Cesarićeva pjesma ima: {} riječi u jednom STIHU.".format(np.round(sum_words / counter)))

U prosjeku, Cesarićeva pjesma ima: 21.0 riječi u jednoj PJESMI.
U prosjeku, Cesarićeva pjesma ima: 4.0 riječi u jednom STIHU.


In [511]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(path)

    def tokenize(self, path):
        
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                if line.count('-') >= 3:
                    continue
                else:
                    words = re.findall(r"[\w']+|[.,!?;:]+", line) + ['<eos>']
                    for word in words:
                        self.dictionary.add_word(word.lower())

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                if line.count('-') >= 3:
                    continue
                else:
                    words = re.findall(r"[\w']+|[.,!?;:]+", line) + ['<eos>']
                    ids = []
                    for word in words:
                        ids.append(self.dictionary.word2idx[word.lower()])
                    idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

### RNN

In [512]:
class RNNModel(nn.Module):
    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for model was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")    
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
            
        self.decoder = nn.Linear(nhid, ntoken)

        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

### Support funkcije

In [513]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    
    return data

In [514]:
def get_batch(source, i):
    seq_len = min(SEQ_LEN, len(source) - 1 - i)
    data = source[i:i + seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [515]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

### Konstante

In [597]:
MODEL = 'LSTM'

EPOCHS = 200

BATCH_SIZE = 21

WORD_EMBEDDINGS = 200
HIDDEN_UNITS_LAYER = 512
N_LAYERS = 2

SEQ_LEN = 21

DROPOUT = 0.5

LEARNING_RATE = 20

SEED = datetime.now().microsecond

WORDS = 500
TEMPERATURE = 1

### Podaci za treniranje

In [523]:
torch.manual_seed(SEED)

corpus = Corpus('./cesaric/train.txt')
train_data = batchify(corpus.train, BATCH_SIZE)

In [524]:
train_data

tensor([[   0,   40,    1,  ...,  209,    1,    7],
        [   1,   17,  356,  ..., 2762,    7,  100],
        [   2,  235,  327,  ..., 2763,  121,    1],
        ...,
        [   7,   13,  657,  ..., 2871,  128, 2321],
        [  69,    7,  115,  ..., 2872, 2227,  205],
        [ 234,  467,    7,  ..., 2873,    1,    7]])

### Model

In [526]:
ntokens = len(corpus.dictionary)
model = RNNModel(MODEL, ntokens, WORD_EMBEDDINGS, HIDDEN_UNITS_LAYER, N_LAYERS, DROPOUT, tie_weights=False)

criterion = nn.NLLLoss()

### Treniranje

In [527]:
learning_rate = LEARNING_RATE

best_loss = None
for epoch in range(1, EPOCHS + 1):
        
    model.train()
    total_loss = 0.
    
    hidden = model.init_hidden(BATCH_SIZE)
    ntokens = len(corpus.dictionary)
    for _, i in enumerate(range(0, train_data.size(0) - 1, SEQ_LEN)):
        data, targets = get_batch(train_data, i)

        model.zero_grad()
        
        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden)
        
        loss = criterion(output, targets)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-learning_rate)

        total_loss += len(data) * loss.item()
    
    loss = total_loss / (len(train_data) - 1)
    
    if not best_loss or loss < best_loss:
        with open('./model/model.pt', 'wb') as f:
            torch.save(model, f)
        best_loss = loss
    else:
        learning_rate /= 4.0

### Generiranje

In [598]:
SEED = datetime.now().microsecond

torch.manual_seed(SEED)

print("Seed: {}".format(SEED))

with open('./model/model.pt', 'rb') as f:
    model = torch.load(f)
model.eval()

corpus = Corpus('./cesaric/train.txt')
ntokens = len(corpus.dictionary)

hidden = model.init_hidden(1)

input = torch.randint(ntokens, (1, 1), dtype=torch.long)

print("Input: {}".format(corpus.dictionary.idx2word[input]))

with open('./output/generirani_cesaric.txt', 'w', encoding = 'utf-8') as outf:
    with torch.no_grad(): 
        for i in range(WORDS):
            output, hidden = model(input, hidden)
            
            word_weights = output.squeeze().div(TEMPERATURE).exp()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]
            
            if word == '<eos>':
                outf.write('\n')
            else:
                if word in '.,!?;:':
                    outf.write(word)
                else:
                    outf.write(' ' + word)

Seed: 602337
Input: preneraženo


**Grubi opis:**  
Model uzima teskt, tokenizira ga - svakoj rijeci pridodaje index, napravi batchove, pusti model da trenira na tim batchovima tako da gleda koja je vjerojatnost da ce se nakon i-te rijeci pojaviti bilo koja rijec iz tog batcha te "uzima" onu koja ima najvecu vjerojatnost.

### Generirani tekst

In [599]:
with open('./output/generirani_cesaric.txt', 'r', encoding = 'utf-8') as outf:
    for word in outf:
        print(word, end = '')

 operete.

 ariju, koju zaboravi grad,
 zapjeva neko tu, u tišini
 svojega okna, u crveno veče,
 kao što pučanin puši u kini
 opijum, kada ga popuše svi,
 svi bogataši i mandarini.

 a žene rade, vječito rade,
 i rade i djecu doje,
 i rijetko i nerado u grad povedu
 dečke i kćerkice svoje,
 a kada kroz bogate ulice idu,
 one se izloga boje.

 jer tamo ima malih mašina
 i lutaka na hrpe,
 i sve je to lijepo, i sve je to ljepše
 no njihova lopta od krpe,
 i samo im želje na oči navru,
 i djeca i matere trpe.

 tramvaj je pred njima petnaest koraka,
 one se boje već prijeći.
 neko ih grubo na ulici gurne,
 one ne vele ni riječi
 davno su one već navikle na to
 da ih se gura i gnječi.

 maleno zvonce na vrat'ma dućana
 nije tek igračka puka.
 stupi l' u dućan, nenajavljen zvukom,
 prosjak il čovjek iz puka,
 mogla bi zgrabiti žemlju sa tezge
 kakva siromašna ruka.

 pred zrcalom, koje iskrivljuje lice,
 kosu po starinski dijele
 djevojke, koje na nezgrapne noge
 navlače čarape bijele
 jeft

### 1. Primjer
 * **Seed**: 892
 * **Input**: sudbini
 * **Temperature**: 0.6

samo nam srce, samo toplo srce,  
i sve je sreća što mi oči vide,  
ati trenuci to su slavoluci  
kroz koje ljubav u trijumfu ide!  
  
  
ma kako uzdiglo se srce,  
klonuti mora, mora pasti.  
sudbino, prije no mi klone,  
o daj mu još jedanput cvasti!  
  
  
još jednom opij ga i digni  
milinom jedne mlade žene,  
još jedne zaljubljene oči  
za ove oči zanesene.  

### 2. Primjer
 * **Seed**: 18254
 * **Input**: kabina
 * **Temperature**: 0.6

i često, tkaju, tkaju  
sve zanose nam žarke,  
sve istine i varke.  


nad svime bog je kronos:  
ko livada! tužno ko skrilo.  
podilazi se, otmi, pocigani!  


gle iza moga stakla sjene:  
nove muškarce, neznane žene.  
  
  
probij se među ljude te  
moderna čerga je coupé.  
  
  
sve ti već uzeh prijatelje,  
tek ti da ostaneš kod želje?  
  

uđi već jednom! što kolebaš?  

### 3. Primjer
 * **Seed**: 425589
 * **Input**: talasom
 * **Temperature**: 0.6

ariju, koju zaboravi grad,  
zapjeva neko tu, u tišini  
svojega okna, u crveno veče,  
kao što pučanin puši u kini  
opijum, kada ga popuše svi,  
svi bogataši i mandarini.  
  
  
a žene rade, vječito rade,  
i rade i djecu doje,  
i rijetko i nerado u grad povedu  
dečke i kćerkice svoje,  
a kada kroz bogate ulice idu,  
one se izloga boje.  

### 4. Primjer
 * **Seed**: 675287
 * **Input**: podrhtava
 * **Temperature**: 0.6

da nisi bog, ni titan,  
a kada je življa tu,  
i svom se suncu zlatno smiješi,  
dok te ne poždere dubina.  
  
  
... i njena mala noga klecnu  
na stubi vagona.  
ona se iznenada lecnu:  
pa što to radi ona?  
  
i još je mogla nogu povuć,  
poslušavši klecaje,  
i slomljena se doma dovuć,  
i opet past u jecaje.  

### 5. Primjer
 * **Seed**: 134619
 * **Input**: ruke
 * **Temperature**: 0.8

svjetalce jedno gori u daljini:  
jedino svjetlo u noćnoj dolini:  
i trepti nježno.  
tako milo  
ko da se smiješi.  
ali gle! već se skrilo.  
ugasilo se. i ko bi ga znao,  
zašto ti bude odjedanput žao.  

### Neki od početnih pokušaja

daljini među vide stara  
jesu voda svjetlost odjednom  
svijet svoju njime jednoga  
dopire neba čuje kuda  
daje jutra tuga između  
--  
nestati dune to petrolejska ,   
svlada prvo nizine il opojnost rad'je   
rado ? , je car pobjednikom san on drugi ne crnom kao i sakriven pa   
kosi . neznane je šuti zgrnu   
ih zgnječenih dan budeći pomagali krevet  